In [1]:
# 1. Import Packages/Modules

import numpy as np
import pandas as pd
from beakerx.object import beakerx
from sklearn.model_selection import train_test_split
import cimcb as cb
beakerx.pandas_display_table() # by default display pandas tables as BeakerX interactive tables
print('All packages successfully loaded')

Using TensorFlow backend.


All packages successfully loaded


In [2]:
# 2. Load Data and Peak Sheet

home = ''  
file = 'MTBLS90.xlsx' 
DataTable,PeakTable = cb.utils.load_dataXL(home + file, DataSheet='Data', PeakSheet='Peak') 

Loadings PeakFile: Peak
Loadings DataFile: Data
Data Table & Peak Table is suitable.
TOTAL SAMPLES: 968 TOTAL PEAKS: 189
Done!


In [3]:
# 3. Get X, and Y

# Select subset of Data for the PLS-DA model
DataTable2 = DataTable[(DataTable.Class == 1) | (DataTable.Class == 0)]

# Create a Binary Y vector for stratifiying the samples
Outcomes = DataTable2['Class']                                  
Y = [1 if outcome == 1 else 0 for outcome in Outcomes]         
Y = np.array(Y)                                                

# Split DataTable2 and Y into train and test (with stratification)
DataTrain, DataTest, Ytrain, Ytest = train_test_split(DataTable2, Y, test_size=0.25, stratify=Y, random_state=40)

# Extract and scale the metabolite data from the DataTable
peaklist = PeakTable['Name']                           
XT = DataTrain[peaklist]                                    
XTlog = np.log(XT)                                          
XTscale = cb.utils.scale(XTlog, method='auto')              
XTknn = cb.utils.knnimpute(XTscale, k=3)                   


In [4]:
# 4. Optimise model hyperparameters 

C_range = np.logspace(-2, 10, 13)
gamma_range = np.logspace(-9, 3, 13)
param_grid = dict(gamma=gamma_range, C=C_range)
 
cv = cb.cross_val.kfold(model=cb.model.SVM,                     
                                X=XTknn,                                
                                Y=Ytrain,                              
                                param_dict=param_grid,
                                folds=5)                                
cv.run() 
cv.plot()

Kfold: 100%|██████████| 169/169 [28:04<00:00,  8.70s/it]


metric changed from 'r2q2' to 'auc' as the model is non-parametric.


Loading BokehJS ...

In [5]:
# 6. Test model
model = cb.model.SVM(C=1, gamma=0.001)
model.train(XTknn, Ytrain)
model.evaluate(cutoffscore=0.5) 

Loading BokehJS ...

In [6]:
# 6. Test model

# Get X, Y
mu, sigma  = cb.utils.scale(XTlog, return_mu_sigma=True) 
peaklist = PeakTable.Name 
XV = DataTest[peaklist].values
XVlog = np.log(XV)
XVscale  = cb.utils.scale(XVlog, method='auto', mu=mu, sigma=sigma)
XVknn = cb.utils.knnimpute(XVscale, k=3)
YVpred = model.test(XVknn)

# Evaluate Ypred against Ytest
evals = [Ytest, YVpred] 
model.evaluate(evals) 


Loading BokehJS ...

In [7]:
# 6. Save tables to excel

# CV full / cv 
table = pd.DataFrame(cv.table)
writer = pd.ExcelWriter("MTBLS90_svm_cv.xlsx")
table.to_excel(writer, index=False)
writer.save()

# Evaluate
table = pd.DataFrame(model.table)
writer = pd.ExcelWriter("MTBLS90_svm_eval.xlsx")
table.to_excel(writer, index=False)
writer.save()